In [ ]:
#https://www.codearmo.com/python-tutorial/options-trading-binomial-pricing-model

In [31]:
import math 
import numpy as np

 Combinatorics formula ${n \choose k} = \frac{n!}{k!(n-k)!}$, where 'n' - number of tryies and 'k' is a value

In [5]:
def comb(n, k):
    return math.factorial(n) / ( math.factorial(k) * math.factorial(n-k))

In [15]:
for i in range(5):
    print(i, "heads", comb(4, i), 'combinations possible')

0 heads 1.0 combinations possible
1 heads 4.0 combinations possible
2 heads 6.0 combinations possible
3 heads 4.0 combinations possible
4 heads 1.0 combinations possible


### Probability and payoff
 Probability of an outcome $= {n \choose k} p^k(1-p)^n-k$
 So the payoff is $\sum_{k=0}^n {n \choose k} p^k(1-p)^{n-k} k$.  For the game where we get 1 USD for each 'heads' out of 4 tryies the fair value equails to the expected payoff.

In [29]:
fair_value = 0 
n= 4 # number of coin flips
for k in range(n+1):
    fair_value += comb(n,k)*0.5**k*0.5**(n-k) * k
    
print("expected payoff: ", fair_value)

expected payoff:  2.0


In [28]:
fair_value = 0 
n= 10 # number of coin flips
for k in range(7,n+1): #after 6 heads we are getting paid 
    fair_value += comb(n,k)*0.5**k*0.5**(n-k) * k
    
print(fair_value)

1.26953125


### 1. Theoretical example of binary option pricing

In this example we make an assumption that the historically observed standart deviation is constant for the underlying asset price. Hence std = $\sigma$ .
Lets denote $u$ and $d$ which represent the upward and downwards moement probabilities of the price of underlying asset. $S_u, S_d$ represent the prices of the stock it if moves up or down by one standart deviation. Also $p$ is the risk free probabily of the stock moving up $(1 + r_{interest})$

$$ p = \frac{r -d}{u - d}$$

In order to determine the $u$ and $d$ with the given assumpions we must evaluate them as following:
* $u = e^{\sigma\sqrt{\Delta t}}$ 
* $d = e^{-\sigma\sqrt{\Delta t}}$
* $r = e^{\sigma r \sqrt{\Delta t}}$

where $\sigma$ is the standart deviation of the stock for a period of time $t$. 

So the value for $p = \frac{e^{\sigma r \sqrt{\Delta t}} -  e^{-\sigma\sqrt{\Delta t}}}{e^{\sigma\sqrt{\Delta t}} -  e^{-\sigma\sqrt{\Delta t}}}$

### 2. Probabilities: 
 $$ p = {N \choose k}p^k(1-p)^{N-k} = {N \choose k}(\frac{e^{r\Delta t - d}}{u - d})^k (1- \frac{e^{r\Delta t - d}}{u - d})^{n-k}$$ 
 
### 3. Value of the call
 
 Value of the call would be represented as the probaility weighted expectation of the $S_{1}- K$
 
 $$C_k = \sum_k^N {N \choose k}p^k(1-p)^{N-k} max(S_0 u^k d^{N-k} - K, 0)$$
 

* where ${N \choose k}p^k(1-p)^{N-k}$ is the probability of an outcome
* $max(S_0 u^k d^{N-k} - K, 0)$ is the value of the call (if $S_1 \leq K$ the value is 0 and if $S_1 > K$ the value is $S_1 - K$)
* $S_1 = S_0 u^k d^{N-k}$ Which accounts for up or down movement of the price


The computed result would be representing: expected payoff from the game = its fair price = call value 


In [71]:
def binary_option_price(S0, N, T, sigma, K, r, type_ = "call"):
    dt = T/N
    u = np.exp( sigma * np.sqrt(dt) ) #compute u, d, p from the equasion  
    d =  np.exp( -sigma * np.sqrt(dt) )
    p = ( np.exp(r*dt) - d) / (u - d)

    cv = 0 #call value
    for k in reversed(range(N+1)):
        ST = S0 * u**k * d ** (N-k)
        prob = comb(N, k) * p ** k * (1-p)**(N-k)
        #print(f"S_1 value: {round(ST,2)} \t S_1 - K: {round(max(ST-K,0),2)}\t Prob :{round(prob,2)}")
        if type_ == "call":
            cv += max(ST - K,0) * prob
        elif type == "put":
            cv += max(K- ST,0) * prob
        else:
            raise ValueError("type_ must be 'call' or 'put'" )
    return cv * np.exp(-r*T)
        
print(f"The fair call value is: {binary_option_price(100, 4, 0.5, 0.4, 105, 0.05)}")

The fair call value is: 10.287301121702995


Lets see if $C_k = \sum_k^N {N \choose k}p^k(1-p)^{N-k} max(S_0 u^k d^{N-k} - K, 0)$ converges when $N \to \inf$

In [79]:
for N in range(1, 1002, 100):
    print(f"N {N}: \t Call Value: {binary_option_price(100, N, 0.5, 0.4, 105, 0.05)}")

N 1: 	 Call Value: 12.798596665812862
N 101: 	 Call Value: 10.240780990502742
N 201: 	 Call Value: 10.225873398289371
N 301: 	 Call Value: 10.208462221938994
N 401: 	 Call Value: 10.220450704587046
N 501: 	 Call Value: 10.223075391787864
N 601: 	 Call Value: 10.221906418461561
N 701: 	 Call Value: 10.219168465339328
N 801: 	 Call Value: 10.215793957453602
N 901: 	 Call Value: 10.216656878435852
N 1001: 	 Call Value: 10.218925064081205
